In [16]:
import pandas as pd
from sklearn.cluster import KMeans
import foursquare
import requests
CLIENT_ID = foursquare.CLIENT_ID
CLIENT_SECRET = foursquare.CLIENT_SECRET
VERSION = '20210501' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [7]:
# dataset preparation
url = "Median Household Income By Zip Code in Los Angeles County, California.html"
dfs = pd.read_html(url)
df_census = pd.read_csv("2010-census-populations-by-zip-code.csv")
geodata = pd.read_csv("us-zip-code-latitude-and-longitude.csv", sep=';')
df_income = dfs[0]
merged_data = df_census.join(df_income.set_index('Zip Code'), on='Zip Code')
dataset = merged_data.loc[merged_data['Estimated Median Income'].notna()]
dataset_geo = dataset.join(geodata.set_index('Zip'), on='Zip Code', how='left')
dataset_geo.drop(columns=[
  'State', 'Timezone', 'Daylight savings time flag', 'geopoint'],
  inplace = True)
prepared_ds = dataset_geo[[
  "Zip Code", "City", "Community", "Estimated Median Income",
  "Longitude", "Latitude", "Total Population", "Median Age",
  "Total Males", "Total Females", "Total Households",
  "Average Household Size"]]
ppds = prepared_ds.drop(prepared_ds[prepared_ds['Estimated Median Income'] ==
                             '---'].index)
final_ds = ppds.dropna(axis=0)
final_ds['Estimated Median Income'] = \
    final_ds['Estimated Median Income'].map(lambda x: x.lstrip('$'))
final_ds['Estimated Median Income'] =\
    final_ds['Estimated Median Income'].str.replace(',','')
final_ds["Estimated Median Income"] =\
    pd.to_numeric(final_ds["Estimated Median Income"])

<ipython-input-7-04ceed1bd9df>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ds['Estimated Median Income'] = \
<ipython-input-7-04ceed1bd9df>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ds['Estimated Median Income'] =\
<ipython-input-7-04ceed1bd9df>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [10]:
final_ds.head()

,Zip Code,City,Community,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size
1,90001,Los Angeles,"Los Angeles (South Los Angeles), Florence-Graham",43360,-118.24878,33.972914,57110,26.6,28468,28642,12971,4.40
2,90002,Los Angeles,"Los Angeles (Southeast Los Angeles, Watts)",37285,-118.24845,33.948315,51223,25.5,24876,26347,11731,4.36
3,90003,Los Angeles,"Los Angeles (South Los Angeles, Southeast Los ...",40598,-118.27600,33.962714,66266,26.3,32631,33635,15642,4.22
4,90004,Los Angeles,"Los Angeles (Hancock Park, Rampart Village, Vi...",49675,-118.30755,34.077110,62180,34.8,31302,30878,22547,2.73
5,90005,Los Angeles,"Los Angeles (Hancock Park, Koreatown, Wilshire...",38491,-118.30848,34.058911,37681,33.9,19299,18382,15044,2.50


In [14]:
areas = final_ds.drop(['Estimated Median Income', 'Total ' 'Population',
                        'Median Age', 'Total Males', 'Total Females',
                        'Total Households', 'Average Household Size'], 1)

In [15]:
areas.head()

,Zip Code,City,Community,Longitude,Latitude
1,90001,Los Angeles,"Los Angeles (South Los Angeles), Florence-Graham",-118.24878,33.972914
2,90002,Los Angeles,"Los Angeles (Southeast Los Angeles, Watts)",-118.24845,33.948315
3,90003,Los Angeles,"Los Angeles (South Los Angeles, Southeast Los ...",-118.27600,33.962714
4,90004,Los Angeles,"Los Angeles (Hancock Park, Rampart Village, Vi...",-118.30755,34.077110
5,90005,Los Angeles,"Los Angeles (Hancock Park, Koreatown, Wilshire...",-118.30848,34.058911


In [ ]:
venues_list = []
for index, area in areas.iterrows():
    url = 'https://api.foursquare' \
      '.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},' \
      '{}&radius={}&limit={}&offset={}&categoryId={}&sortByPopularity={}'\
        .format(
    CLIENT_ID, CLIENT_SECRET, VERSION, area['Latitude'], area['Longitude'],
    1000, LIMIT, 0, '4d4b7105d754a06374d81259', 1)

    results = requests.get(url).json()["response"]['groups'][0]['items']
    for v in results:
        try:
            city = v['venue']['location']['city']
        except:
            city = area['City']

        try:
            postalCode = str(v['venue']['location']['postalCode'])
        except:
            postalCode = str(area['Zip Code'])

        if postalCode == str(area['Zip Code']):
            venues_list.append((
                        area['Zip Code'],
                        area['Community'],
                        area['Latitude'],
                        area['Longitude'],
                        v['venue']['name'],
                        v['venue']['categories'][0]['name'],
                        city ))

la_venues = pd.DataFrame(venues_list, columns=['Zip Code', 'Community',
              'Zip Code Latitude', 'Zip Code Longitude', 'Venue',
              'Venue Category', 'City'])

la_venues.count()

In [ ]:
# la_venues sortieren 

In [ ]:
import plotly.express as px
fig = px.bar(la_venues, x="Venue Category", y="Count", title='Distribution '
                                                             'of '
                                                             'recommended Food'
                                                             ' Venues')
fig.show()
